![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_NLLB.ipynb)

# Import OpenVINO NLLB models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and importing NLLB models from HuggingFace  for use in Spark NLP, with [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html). The focus is on converting the model to the OpenVINO format and applying precision optimizations (INT8 and INT4), to enhance the performance and efficiency on CPU platforms using [Optimum Intel](https://huggingface.co/docs/optimum/main/en/intel/inference).

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance CPU inference for models. So please make sure you have upgraded to the latest Spark NLP release.
- Model quantization is a computationally expensive process, so it is recommended to use a runtime with more than 32GB memory for exporting the quantized model from HuggingFace.
- You can import NLLB models via `NLLBModel`. These models are usually under `Text Generation` category and have `NLLB` in their labels.
- Some [example models](https://huggingface.co/models?search=NLLB)

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future release, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00


[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API. It also offers the ability to perform weight compression during export.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [openbmb/NLLB-2B-dpo-bf16](https://huggingface.co/openbmb/NLLB-2B-dpo-bf16) model from HuggingFace as an example.
- In addition to `NLLBModel` we also need to save the tokenizer. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

### Option 1: Exporting to OpenVINO IR in INT8 Precision

Passing the `load_in_8bit` parameter applies 8-bit quantization on the model weights.

In [2]:
from optimum.intel.openvino import OVQuantizer, OVModelForCausalLM, OVModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoConfig

MODEL_NAME = "facebook/nllb-200-distilled-600M"
EXPORT_PATH = f"./models/int8/{MODEL_NAME}"

model = OVModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME, export=True, load_in_8bit=True, trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = AutoConfig.from_pretrained(MODEL_NAME)

model.save_pretrained(f"./models/int8/{MODEL_NAME}")
tokenizer.save_pretrained(f"./models/int8/{MODEL_NAME}/assets")
config.save_pretrained(f"./models/int8/{MODEL_NAME}/assets")

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

...

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_asym                 │ 100% (122 / 122)            │ 100% (122 / 122)                       │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_asym                 │ 100% (98 / 98)              │ 100% (98 / 98)                         │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 200}
  warnings.warn(


In [3]:
import json
import os

vocab = tokenizer.get_vocab()
vocab_path = os.path.join("models", "int8", MODEL_NAME, "assets", "vocab.json")

with open(vocab_path, "w") as f:
    json.dump(vocab, f)

### Option 2: Exporting to OpenVINO IR in INT4 Precision

Alternately, Optimum Intel also provides [4-bit weight compression](https://huggingface.co/docs/optimum/intel/optimization_ov#4-bit) with `OVWeightQuantizationConfig` class to control weight quantization parameters. The `ratio` parameter controls the ratio between 4-bit and 8-bit quantization. If set to 0.8, it means that 80% of the layers will be quantized to int4 while the remaining 20% will be quantized to int8.

In [2]:
from optimum.intel.openvino import OVWeightQuantizationConfig, OVModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoConfig

MODEL_NAME = "facebook/nllb-200-distilled-600M"
EXPORT_PATH = f"./models/int4/{MODEL_NAME}"


q_config = OVWeightQuantizationConfig(bits=4, sym=True, group_size=128, ratio=0.8)

model = OVModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME, export=True, quantization_config=q_config, trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
config = AutoConfig.from_pretrained(MODEL_NAME, trust_remote_code=True)

model.save_pretrained(f"./models/int4/{MODEL_NAME}")
tokenizer.save_pretrained(f"./models/int4/{MODEL_NAME}/assets")
config.save_pretrained(f"./models/int4/{MODEL_NAME}/assets")


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

...

Output()

Output()

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│ Weight compression mode   │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│ int8_asym                 │ 68% (36 / 98)               │ 20% (34 / 96)                          │
├───────────────────────────┼─────────────────────────────┼────────────────────────────────────────┤
│ int4_sym                  │ 32% (62 / 98)               │ 80% (62 / 96)                          │
┕━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 200}
  warnings.warn(


In [3]:
import json
import os

vocab = tokenizer.get_vocab()
vocab_path = os.path.join("models", "int4", MODEL_NAME, "assets", "vocab.json")

with open(vocab_path, "w") as f:
    json.dump(vocab, f)

### Prepare assests

Once the model export and quantization is complete, move the model assets needed for tokenization in Spark NLP to the `assets` directory.

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {EXPORT_PATH}

total 1097708
drwxr-xr-x 2 root root      4096 Jun 23 03:22 assets
-rw-r--r-- 1 root root       897 Jun 23 03:22 config.json
-rw-r--r-- 1 root root       184 Jun 23 03:22 generation_config.json
-rw-r--r-- 1 root root       609 Jun 23 03:22 openvino_config.json
-rw-r--r-- 1 root root 388856668 Jun 23 03:22 openvino_decoder_model.bin
-rw-r--r-- 1 root root    994890 Jun 23 03:22 openvino_decoder_model.xml
-rw-r--r-- 1 root root 373227316 Jun 23 03:22 openvino_decoder_with_past_model.bin
-rw-r--r-- 1 root root    850931 Jun 23 03:22 openvino_decoder_with_past_model.xml
-rw-r--r-- 1 root root 359534404 Jun 23 03:22 openvino_encoder_model.bin
-rw-r--r-- 1 root root    568461 Jun 23 03:22 openvino_encoder_model.xml


In [5]:
!ls -l {EXPORT_PATH}/assets

total 43456
-rw-r--r-- 1 root root      896 Jun 23 03:22 config.json
-rw-r--r-- 1 root root  4852054 Jun 23 03:22 sentencepiece.bpe.model
-rw-r--r-- 1 root root     4232 Jun 23 03:22 special_tokens_map.json
-rw-r--r-- 1 root root    39891 Jun 23 03:22 tokenizer_config.json
-rw-r--r-- 1 root root 32240136 Jun 23 03:22 tokenizer.json
-rw-r--r-- 1 root root  7347254 Jun 23 03:22 vocab.json


## 2. Import and Save NLLB in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 23.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `NLLBTransformer` which allows us to load the OpenVINO model.
- Most params will be set automatically. They can also be set later after loading the model in `NLLBTransformer` during runtime, so don't worry about setting them now.
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

let's load the int4 model

In [8]:
from sparknlp.annotator import NLLBTransformer

NLLB = NLLBTransformer \
    .loadSavedModel(EXPORT_PATH, spark) \
    .setMaxOutputLength(50) \
    .setInputCols(["documents"]) \
    .setOutputCol("generation")

Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
NLLB.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your OpenVINO NLLB model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 737248
drwxr-xr-x 4 root root      4096 Jun 23 03:25 fields
drwxr-xr-x 2 root root      4096 Jun 23 03:25 metadata
-rw-r--r-- 1 root root   4852054 Jun 23 03:26 nllb_spp
-rw-r--r-- 1 root root 389911358 Jun 23 03:26 openvino_decoder_model.xml
-rw-r--r-- 1 root root 360158130 Jun 23 03:26 openvino_encoder_model.xml


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny NLLB model 😊

In [12]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import NLLBTransformer
from pyspark.ml import Pipeline

test_data = spark.createDataFrame([
    [1, """生活就像一盒巧克力""".strip().replace("\n", " ")]
]).toDF("id", "text")

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

NLLB = NLLBTransformer \
    .load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(50) \
    .setOutputCol("generation") \
    .setSrcLang("zho_Hans") \
    .setTgtLang("eng_Latn")

pipeline = Pipeline().setStages([
    document_assembler,
    NLLB
])

model = pipeline.fit(test_data)
results = model.transform(test_data)

results.select("generation.result").show(truncate=False)

+---------------------------------+
|result                           |
+---------------------------------+
|[Life is like a box of chocolate]|
+---------------------------------+



That's it! You can now go wild and use hundreds of NLLB models from HuggingFace 🤗 in Spark NLP 🚀
